# Consolidating Route Patterns in GTFS

Occurs after running R script that filters the trips, schedules, etc. by a chosen date.
This part first came after an import to TransCAD but this is an alternate approach of consolidating solely based on GTFS instead of trying to join the calculated trips per Route_Name back to the route_pattern_id. There isn't a great field to join back on as we don't know exactly how TransCAD is translating the GTFS information into the Route_Names and how the routes are being consolidated as the number of unique route patterns differs between TransCAD and GTFS with GTFS having less unique route patterns in its trip table *(both post R filtering)

As discussed with Marty and Sabiheh, the goal of this consolidation step is to re-assign route patterns with low numbers of trips across the day ( less than three in each time period) to route_pattern_ids with the most trips within their matching Route, Direction, and Headsign. This gets filtered down to the trip level where if a trip's route pattern is consolidated, it will be replaced with the route_pattern_id with max trips.

All trips with same headsign/route/direction  are assigned the same route_pattern_id. The route_pattern_id that represents the group is the route_pattern_id with the most trips per day.

If less than 3 trips per tod period across all day, assign the route_pattern_id of the dir/route group with the most trips.

Calc TOD by 3-3 not 6:30 - 30:29

reorder stop times at end by trip_id

In [1]:
import time
import datetime

import matplotlib
matplotlib.use('agg')  # allows notebook to be tested in Travis

import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import pandana as pdna
import time

import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network

%matplotlib inline

In [6]:
# required bbox including all of Massachusetts and RI as well as parts of NH, CT, NY
bbox = (-73.7207, 41.1198, -69.7876, 43.1161)
# path to the downloaded and cleaned gtfs - mbta recap file for fall 2018
#   this could also be a folder of gtfs folders (pre merge of multiple gtfs)

path_to_gtfs = r"C:\Users\matkinson.AD\Downloads\Nov12_Sandbox\Part_2_GTFS_R\mbta2018_102418_20221109"

In [7]:
loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(gtfsfeed_path= path_to_gtfs,
                                           validation=True,
                                           verbose=True,
                                           bbox=bbox,
                                           remove_stops_outsidebbox=False,
                                           append_definitions=True)

Checking GTFS text file header whitespace... Reading files using encoding: utf-8 set in configuration.
GTFS text file header whitespace check completed. Took 0.12 seconds
--------------------------------
Processing GTFS feed: mbta2018_102418_20221109
The unique agency id: mbta was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.03 seconds
Unique GTFS feed id operation complete. Took 0.00 seconds
No GTFS feed stops were found to be outside the bounding box coordinates
mbta2018_102418_20221109 GTFS feed stops: coordinates are in northwest hemisphere. Latitude = North (90); Longitude = West (-90).
Appended route type to stops
Appended route type to stop_times
--------------------------------
Added descriptive definitions to stops, routes, stop_times, and trips tables
Successfully converted ['departure_time'] to seconds past midnight and appended new columns to stop_times. Took 1.01 seconds
1 GTFS feed file(s) successfully read as 

Needs/Steps:
- Number of trips per time period per route_pattern_id
    - Midpoint time of each trip
    - Each trip classified by TOD (based on midpoint)
    - Sum of trips per TOD by route_pattern_id
- route_pattern_id with most daily trips per Route, Direction, Headsign
    - Sum all tod trips per route_pattern_id
    - grab just the max per Route, Direction, Headsign (but keep route_pattern_id)
- consolidate route patterns by Route, Direction, Headsign
    - if route_pattern_id has less than 3 trips in each of the 4 TODs, replace with max trips route_pattern_id

In [8]:
def get_start_stop_times(stop_times):    
    chocula =0 
    for trip_id in stop_times['trip_id'].unique():
        max_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.max()')[['trip_id','arrival_time']]
        min_row = stop_times.query('trip_id==@trip_id').query('stop_sequence == stop_sequence.min()')[['trip_id','arrival_time']]
        r2 = min_row.merge(max_row, how='left', on='trip_id', suffixes = ('_start','_end'))
        if chocula == 0:
            flintstone = pd.DataFrame(r2)
        else:
            flintstone=pd.concat([flintstone,r2])
        chocula +=1
    return(flintstone)


In [9]:
simpson = get_start_stop_times(gtfsfeeds_dfs.stop_times)

#### Check the results! 

In [ ]:
simpson['arrival_time_end'].str.split(':').str[0]

In [ ]:
simpson.query('arrival_time_end.str.split(":").str[0].astype("int32") > 23')

In [ ]:
simpson.query('arrival_time_start.str.split(":").str[0].astype("int32") < 6').sort_values(by='arrival_time_start')

In [ ]:
((simpson['arrival_time_end'].str.split(":").str[1]).astype('int32')/60)

#### Start work again!

In [10]:
def assign_tod(start_stop):
    start_stop['at_end_dec'] = (
        (
            (start_stop['arrival_time_end'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_end'].str.split(":").str[1]
            ).astype('int32')/60)))
    start_stop['at_start_dec'] = (
        (
            (start_stop['arrival_time_start'].str.split(":").str[0]).astype('int32')
            +
            ((start_stop['arrival_time_start'].str.split(":").str[1]
            ).astype('int32')/60)))
    
    start_stop['midpoint'] = start_stop['at_start_dec'] + ((start_stop['at_end_dec']-start_stop['at_start_dec'])/2)
    start_stop['tod'] = np.where(start_stop['midpoint'].between(6.50,9.50),'AM', np.where(
        start_stop['midpoint'].between(9.50,15.00), 'MD', np.where(
            start_stop['midpoint'].between(15.00,19.00),'PM', 'NT' 
        )
            ) 
        )
    
    return start_stop


In [11]:
smurf = assign_tod(simpson)

In [12]:
smurf

,trip_id,arrival_time_start,arrival_time_end,at_end_dec,at_start_dec,midpoint,tod
0,37940074,19:25:00,20:03:00,20.050000,19.416667,19.733333,NT
0,37940075,06:35:00,07:03:00,7.050000,6.583333,6.816667,AM
0,37940076,07:14:00,07:53:00,7.883333,7.233333,7.558333,AM
0,37940077,11:12:00,11:50:00,11.833333,11.200000,11.516667,MD
0,37940079,05:48:00,06:25:00,6.416667,5.800000,6.108333,NT
...,...,...,...,...,...,...,...
0,CR-Weekday-Fall-18-924,20:35:00,21:08:00,21.133333,20.583333,20.858333,NT
0,CR-Weekday-Fall-18-925,21:40:00,22:19:00,22.316667,21.666667,21.991667,NT
0,CR-Weekday-Fall-18-926,22:35:00,23:10:00,23.166667,22.583333,22.875000,NT
0,CR-Weekday-Fall-18-927,22:40:00,23:20:00,23.333333,22.666667,23.000000,NT


#### Exploration: For Route/Headsign/Direction ids with multiple route_pattern_ids, does the TOD period matter?
- Service_ID is not something that seems to differentiate why we would have multiple route_pattern_ids.
- Yes TOD does matter, see below: 
    - Basically, a route might run one way in the morning and a different way in the afternoon. See Route 704 for a great example. These routes do not deviate enough to be a new headsign or route (no VIA either) but may take a different road for a short while. This likely affects a few stops but not enough to change the headsign as previously noted.

In [68]:
# FOR R/H/D, select RPID with most trips
tod_trips = gtfsfeeds_dfs.trips.merge(smurf[['trip_id','tod']], how='left', on='trip_id')
day_rpid = tod_trips.groupby(by=['route_pattern_id']).agg({'trip_id':'nunique', 'route_id':'first','trip_headsign':'first','direction_id':'first'})
day_rpid = day_rpid.rename(columns = {'trip_id':'daily_trips'}).reset_index()
max_rpid = day_rpid.groupby(by=['route_id','trip_headsign','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
max_rpid = max_rpid[['route_id','trip_headsign','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})
fred = max_rpid[max_rpid.duplicated(subset=['route_id','trip_headsign','direction_id'], keep =False)]
max_rpid[max_rpid.duplicated(subset=['route_id','trip_headsign','direction_id'], keep =False)]

,route_id,trip_headsign,direction_id,route_pattern_id_new
78,134,Wellington via Veterans Senior Center,1,134-1-1
79,134,Wellington via Veterans Senior Center,1,134-2-1
102,16,Forest Hills,0,16-2-0
103,16,Forest Hills,0,16-3-0
156,217,Quincy Center via North Quincy,0,217-1-0
157,217,Quincy Center via North Quincy,0,217-8-0
205,240,Quincy Center via North Randolph,1,240-2-1
206,240,Quincy Center via North Randolph,1,240-3-1
292,36,Forest Hills,1,36-1-1
293,36,Forest Hills,1,36-_-1


In [90]:
# OG
tod_trips = gtfsfeeds_dfs.trips.merge(smurf[['trip_id','tod']], how='left', on='trip_id')
tod_rpid = tod_trips.groupby(by=['route_pattern_id','tod']).agg({'tod':'count', 'route_id':'first','trip_headsign':'first','direction_id':'first'})
tod_rpid = tod_rpid.rename(columns = {'tod':'trips_per_tod'}).reset_index()

george = fred.merge(tod_rpid, how='right', left_on='route_pattern_id_new',right_on='route_pattern_id').query('~route_id_x.isna()')
george.query('route_id_x == "70A"').sort_values(by=['trip_headsign_x'])

,route_id_x,trip_headsign_x,direction_id_x,route_pattern_id_new,route_pattern_id,tod,trips_per_tod,route_id_y,trip_headsign_y,direction_id_y
1560,70A,North Waltham,0.0,70A-1-0,70A-1-0,MD,3,70A,North Waltham,0
1561,70A,North Waltham,0.0,70A-1-0,70A-1-0,PM,7,70A,North Waltham,0
1564,70A,North Waltham,0.0,70A-4-0,70A-4-0,AM,6,70A,North Waltham,0
1565,70A,North Waltham,0.0,70A-4-0,70A-4-0,MD,2,70A,North Waltham,0
1566,70A,North Waltham,0.0,70A-4-0,70A-4-0,NT,2,70A,North Waltham,0
1562,70A,University Park,1.0,70A-1-1,70A-1-1,AM,5,70A,University Park,1
1563,70A,University Park,1.0,70A-1-1,70A-1-1,MD,5,70A,University Park,1
1567,70A,University Park,1.0,70A-4-1,70A-4-1,MD,2,70A,University Park,1
1568,70A,University Park,1.0,70A-4-1,70A-4-1,NT,2,70A,University Park,1
1569,70A,University Park,1.0,70A-4-1,70A-4-1,PM,6,70A,University Park,1


#### Actually update the route_pattern_ids
- Getting duplicate ids where same number of trips for max, so choosing one arbitrarily.
- The above section can be expanded to try to determine which has the most trips during peak period, but for now this works.

In [85]:
# FOR R/H/D, select RPID with most trips
tod_trips = gtfsfeeds_dfs.trips.merge(smurf[['trip_id','tod']], how='left', on='trip_id')
day_rpid = tod_trips.groupby(by=['route_pattern_id']).agg({'trip_id':'nunique', 'route_id':'first','trip_headsign':'first','direction_id':'first'})
day_rpid = day_rpid.rename(columns = {'trip_id':'daily_trips'}).reset_index()
max_rpid = day_rpid.groupby(by=['route_id','trip_headsign','direction_id']).apply(lambda g: g[g['daily_trips'] == g['daily_trips'].max()])[['route_pattern_id','daily_trips']].reset_index()
max_rpid = max_rpid[['route_id','trip_headsign','direction_id','route_pattern_id']].rename(columns = {'route_pattern_id':'route_pattern_id_new'})

max_rpid = max_rpid.drop_duplicates(subset=['route_id','trip_headsign','direction_id'])
max_rpid

,route_id,trip_headsign,direction_id,route_pattern_id_new
0,1,Dudley,1,1-_-1
1,1,Harvard,0,1-_-0
2,10,Andrew via South Bay Center,0,10-6-0
3,10,City Point,0,10-_-0
4,10,City Point via South Bay Center,0,10-9-0
...,...,...,...,...
703,Orange,Forest Hills,0,Orange-3-0
704,Orange,Oak Grove,1,Orange-3-1
705,Red,Alewife,1,Red-3-1
706,Red,Ashmont,0,Red-1-0


In [56]:
trips_update1 = gtfsfeeds_dfs.trips.merge(max_rpid, how='left', on=['route_id','trip_headsign','direction_id'])
trips_update1['route_pattern_id'] = trips_update1['route_pattern_id_new']
#trips_update1 = trips_update1[gtfsfeeds_dfs.trips.columns]
trips_update1[trips_update1.duplicated(subset=['trip_id'], keep =False)]

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed,unique_agency_id,unique_feed_id,bikes_allowed_desc,wheelchair_accessible_desc,route_pattern_id_new
2070,134,BUS42018-hbf48011-Weekday-02,38428492,Wellington via Veterans Senior Center,NaN,1,F134-70,1340174,1,NaN,134-1-1,1,mbta,mbta2018_102418_20221109_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,134-1-1
2071,134,BUS42018-hbf48011-Weekday-02,38428492,Wellington via Veterans Senior Center,NaN,1,F134-70,1340174,1,NaN,134-2-1,1,mbta,mbta2018_102418_20221109_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,134-2-1
2072,134,BUS42018-hbf48011-Weekday-02,38428504,Wellington via Veterans Senior Center,NaN,1,F134-66,1340172,1,NaN,134-1-1,1,mbta,mbta2018_102418_20221109_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,134-1-1
2073,134,BUS42018-hbf48011-Weekday-02,38428504,Wellington via Veterans Senior Center,NaN,1,F134-66,1340172,1,NaN,134-2-1,1,mbta,mbta2018_102418_20221109_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,134-2-1
2416,16,BUS42018-hbc48wk1-Weekday-02,38229839,Forest Hills,NaN,0,C16-144,160142,1,NaN,16-2-0,1,mbta,mbta2018_102418_20221109_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,16-2-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15432,CR-Newburyport,CR-Weekday-Newburyport-Fall-18,CR-Weekday-Fall-18-128,North Station,128.0,1,NaN,9810006,1,NaN,CR-Newburyport-1-1,1,mbta,mbta2018_102418_20221109_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,CR-Newburyport-1-1
15433,CR-Newburyport,CR-Weekday-Newburyport-Fall-18,CR-Weekday-Fall-18-192,North Station,192.0,1,NaN,9810006,1,NaN,CR-Newburyport-0-1,2,mbta,mbta2018_102418_20221109_1,no bicycles are allowed on this trip,can accommodate at least one rider in a wheelc...,CR-Newburyport-0-1
15434,CR-Newburyport,CR-Weekday-Newburyport-Fall-18,CR-Weekday-Fall-18-192,North Station,192.0,1,NaN,9810006,1,NaN,CR-Newburyport-1-1,2,mbta,mbta2018_102418_20221109_1,no bicycles are allowed on this trip,can accommodate at least one rider in a wheelc...,CR-Newburyport-1-1
15435,CR-Newburyport,CR-Weekday-Newburyport-Fall-18,CR-Weekday-Fall-18-194,North Station,194.0,1,NaN,9810006,1,NaN,CR-Newburyport-0-1,1,mbta,mbta2018_102418_20221109_1,can accommodate at least one bicycle,can accommodate at least one rider in a wheelc...,CR-Newburyport-0-1


In [18]:
tod_trips = gtfsfeeds_dfs.trips.merge(smurf[['trip_id','tod']], how='left', on='trip_id')
tod_rpid = tod_trips.groupby(by=['route_id','trip_headsign','direction_id','tod']).agg({'tod':'count','route_pattern_id':'nunique','service_id':'nunique'})
tod_rpid = tod_rpid.rename(columns = {'tod':'trips_per_tod', 'route_pattern_id':'unique_rpid'}).reset_index()
tod_rpid

,route_id,trip_headsign,direction_id,tod,trips_per_tod,unique_rpid,service_id
0,1,Dudley,1,AM,18,1,1
1,1,Dudley,1,MD,24,1,1
2,1,Dudley,1,NT,39,1,1
3,1,Dudley,1,PM,28,1,1
4,1,Harvard,0,AM,20,1,1
...,...,...,...,...,...,...,...
1936,Red,Ashmont,0,PM,25,1,1
1937,Red,Braintree,0,AM,22,1,1
1938,Red,Braintree,0,MD,29,1,1
1939,Red,Braintree,0,NT,35,1,1


In [27]:
tod_rpid.query('unique_rpid > 2 & service_id >= 2')

,route_id,trip_headsign,direction_id,tod,trips_per_tod,unique_rpid,service_id
223,134,Wellington,1,NT,10,3,2
279,16,Forest Hills,0,AM,15,3,2
598,28,Mattapan,0,MD,34,4,3
1213,60,Chestnut Hill Mall,0,NT,11,3,2
1408,73,Harvard,1,AM,30,3,2


In [26]:
#tod_trips.query('route_id == "10" & tod == "AM" & direction_id == 0')

In [ ]:
byrpid = tod_rpid.groupby(
    by=['route_id','trip_headsign','direction_id','route_pattern_id']).sum(
        'trips_per_tod').reset_index().rename(columns={'trips_per_tod':'daily_trips'})

gby = byrpid.set_index(['route_id','direction_id','trip_headsign'])
gby.index = ['{}_{}_{}'.format(i, j, k) for i, j, k in gby.index]
chocula = 0
for name in gby.index:
    if len(pd.DataFrame(gby.loc[name,:]).transpose()) > 1:
        max_row = pd.DataFrame(gby.loc[name,:]).query('daily_trips == daily_trips.max()')
    else:
        max_row = pd.DataFrame(gby.loc[name,:]).transpose().query('daily_trips == daily_trips.max()')
    if chocula == 0:
        flintstone = pd.DataFrame(max_row)
    else:
        flintstone=pd.concat([flintstone,max_row])
    chocula +=1

In [ ]:
woolf = tod_rpid.pivot_table(index = ['route_pattern_id','route_id','trip_headsign','direction_id'], columns = ['tod'])

needs_update = woolf['trips_per_tod'].reset_index().fillna(0).query('AM < 3 & MD < 3 & PM < 3 & NT < 3')
needs_update = needs_update.set_index(['route_id','direction_id','trip_headsign'])
needs_update.index = ['{}_{}_{}'.format(i, j, k) for i, j, k in needs_update.index]
needs_update.reset_index()

In [ ]:
flintstone.reset_index()

In [ ]:
update_table = needs_update.reset_index().merge(flintstone.reset_index(), how='left', on='index', suffixes = ('_old','_max')).drop_duplicates('route_pattern_id_old')
update_table

#### Quick break to calculate some numbers per route pattern id before they are updated!

In [ ]:
smurf['trip_length'] = smurf['at_end_dec'] - smurf['at_start_dec']
smurf['trip_length_time'] = smurf['trip_length'].astype('int32').astype('str') + ":" + ((smurf['trip_length'] % 1)*60).astype('int32').astype('str') + ":00"
smurf

In [ ]:
papa_smurf = smurf.merge(gtfsfeeds_dfs.trips[['trip_id','route_pattern_id']], how='left', on='trip_id')
smurfette = papa_smurf.sort_values('midpoint').groupby(by=['route_pattern_id','tod']).agg({'trip_length':['mean','median','count','first','last']})
smurfette = smurfette.droplevel(0,axis=1).reset_index()
smurfette['first_last_dif'] = round(smurfette['first'] - smurfette['last'],2)
smurfette['first_last_dif_min'] = smurfette['first_last_dif']*60
smurfette

#### Break over - update those route_pattern_ids!

In [ ]:
tripstrips = gtfsfeeds_dfs.trips.merge(update_table[['route_pattern_id_old','route_pattern_id_max']], how='left', left_on='route_pattern_id', right_on='route_pattern_id_old')

tripstrips['route_pattern_id'] = np.where(tripstrips['route_pattern_id_old'].isna(), tripstrips['route_pattern_id'], tripstrips['route_pattern_id_max'])

tripstrips

In [ ]:
tripstrips['trip_ids']

In [ ]:
len(tripstrips['route_pattern_id'].unique())

In [ ]:
len(gtfsfeeds_dfs.trips['route_pattern_id'].unique())

In [ ]:
gtfsfeeds_dfs.trips = tripstrips

### Create Generic Stop Times & Stop Sequence per Route Pattern

- Can't just change route_pattern_id as TransCAD does not use this field to combine trips into routes. There is no effect on the import.
- Working theory is that to consolidate routes one must use the stop_times.txt table as it defines the stop sequence for every trip. Theoretically, this is being used to consolidate trips into routes based on whether they have the same stop sequence.

Plan:
- Explore if stop times differ depending on TOD or if only realtime GTFS takes into account traffic.
    - For every route_pattern_id, get the average trip length (in minutes).
- For every route_pattern_id, get the average number of minutes between each pair of stops in the stop sequence.
- Replace the stop times and sequence for trips that had their route_pattern_id updated with the generic stop sequence and times per route_pattern_id created in the previous step. 
    - Keep the start time and work off of that.
    - Arrival time will equal departure time given that the difference is usually less than a minute. Will assume difference in time can be included in the minutes to next arrival time for aggregate modeling purposes.

In [ ]:
trip_stop_replace = {}

gtfsfeeds_dfs.stop_times['first_stop'] =  0
gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.groupby('trip_id').stop_sequence.idxmin(),'first_stop'] = 1

for idx, row in gtfsfeeds_dfs.trips.query('(~route_pattern_id_old.isna()) & (route_pattern_id_old != route_pattern_id_max)').iterrows():
    tid = row['trip_id']
    rpid = gtfsfeeds_dfs.trips.query('trip_id == @tid').route_pattern_id

    all_trips = gtfsfeeds_dfs.trips.query('route_pattern_id == @rpid.iloc[0]').trip_id

    start_time = gtfsfeeds_dfs.stop_times.query('(trip_id == @tid) & (first_stop == 1)')
    all_start_times = gtfsfeeds_dfs.stop_times.query('(trip_id in @all_trips) & (first_stop == 1)')

    test_list = [[x,(abs(start_time['departure_time_sec']- all_start_times.query('trip_id == @x'))['departure_time_sec'].iloc[0])] for x in all_start_times['trip_id']]
    close = {}
    close = {sub[0]:sub[1] for sub in test_list}
        
    min_t = min(close, key=close.get)
    if tid != min_t:
        trip_stop_replace[tid] = min_t

In [ ]:
gtfsfeeds_dfs.stop_times = gtfsfeeds_dfs.stop_times.sort_values('stop_sequence')

for tid in trip_stop_replace.values():
    
    gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.loc[:,'trip_id'] == str(tid), 'time_between_stops'] = gtfsfeeds_dfs.stop_times.loc[gtfsfeeds_dfs.stop_times.loc[:,'trip_id'] == str(tid), 'departure_time_sec'].diff()

gtfsfeeds_dfs.stop_times

In [ ]:
gtfsfeeds_dfs.stop_times.query('time_between_stops > 0')

In [ ]:
stop_times = gtfsfeeds_dfs.stop_times
for trip in trip_stop_replace.keys():
    start_time = stop_times.query('(trip_id == @trip) & (first_stop == 1)')['departure_time_sec']
    # drop old stop times
    stop_times = stop_times.drop(
        stop_times.loc[stop_times['trip_id']==trip].index)
    # grab new stop times
    new_trip = trip_stop_replace[trip]
    nst = stop_times.query('trip_id == @new_trip')
    nst['trip_id'] = trip

    # replace the start time, then calculate the stop times by the departure_time_sec difference
    nst.loc[nst.loc[:,'first_stop']==1,'departure_time_sec'] = int(start_time.iloc[0])
    nst.loc[nst.loc[:,'first_stop']==1,'time_between_stops'] = int(start_time.iloc[0])
    nst['departure_time_sec'] = nst['time_between_stops'].cumsum()

    # recalc arrival/dep times
    nst['arrival_time'] = pd.to_datetime(nst['departure_time_sec'],unit='s').astype('str').str[11:19]
    nst['departure_time'] = nst['arrival_time']

    #keep only relevant columns
    nst = nst[stop_times.columns]

    stop_times = pd.concat([stop_times,nst])

In [ ]:
gtfsfeeds_dfs.stop_times = stop_times[['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'timepoint',
       'checkpoint_id', 'unique_agency_id', 'unique_feed_id', 'route_type',
       'pickup_type_desc', 'drop_off_type_desc', 'timepoint_desc',
       'departure_time_sec']]


In [ ]:
gtfsfeeds_dfs.trips.to_csv(r"C:\Users\matkinson.AD\Downloads\sandbox_vetday\trips.txt",index=False)
gtfsfeeds_dfs.stop_times.to_csv(r"C:\Users\matkinson.AD\Downloads\sandbox_vetday\stop_times.txt",index=False)